In [4]:
from langchain_community.document_loaders import TextLoader # type: ignore
loader=TextLoader("speech.txt")
text_documents=loader.load
text_documents

<bound method BaseLoader.load of <langchain_community.document_loaders.text.TextLoader object at 0x00000230C9F98590>>

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()



True

In [6]:
#web based loader
from langchain_community.document_loaders import WebBaseLoader
import bs4
## load , chunck and index
loader=WebBaseLoader(web_paths=("https://vinkjobs.com/welocalize-hiring-for-work-from-home/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post_content","post-header")
                     )))
text_documents=loader.load()


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
text_documents

[Document(metadata={'source': 'https://vinkjobs.com/welocalize-hiring-for-work-from-home/'}, page_content='Sprinto hiring for Full Stack InternSprinto hiring for Recruitment Intern – RemoteNxtWave hiring for Talent Acquisition Specialist')]

In [9]:
print(docs)

[Document(metadata={'producer': 'Antenna House PDF Output Library 7.1.1639', 'creator': 'AH CSS Formatter V7.1 MR2 for Linux64 : 7.1.3.50324 (2021-04-26T09:47+09)(Jisupdf Editor)', 'creationdate': '2022-06-22T13:50:38+00:00', 'author': 'Reis, Joe;Housley, Matt;', 'ebx_publisher': "O'Reilly Media, Incorporated", 'icnappname': 'Jisupdf Editor', 'icnappplatform': 'Windows', 'icnappversion': '2.03', 'moddate': 'D:20240420165355', 'title': 'Fundamentals of Data Engineering', 'source': 'Fundamentals of Data Engineering (Reis, JoeHousley, Matt) (Z-Library).pdf', 'total_pages': 445, 'page': 0, 'page_label': 'Cover'}, page_content='Joe Reis & \n Matt Housley\nFundamentals \nof Data \nEngineering\nPlan and Build \nRobust Data Systems\nReis & \n Housley'), Document(metadata={'producer': 'Antenna House PDF Output Library 7.1.1639', 'creator': 'AH CSS Formatter V7.1 MR2 for Linux64 : 7.1.3.50324 (2021-04-26T09:47+09)(Jisupdf Editor)', 'creationdate': '2022-06-22T13:50:38+00:00', 'author': 'Reis, Jo

In [21]:
# vecter embedding and vector store
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
embedding = OllamaEmbeddings(model="nomic-embed-text")
db=Chroma.from_documents(documents[:100],embedding)


In [22]:
## Chroma vector
query="what is data engineering"
result=db.similarity_search(query)
result[0].page_content

'rise of data science in the 2010s. For the purpose of this book, it’s critical to define\nwhat data engineering and data engineer mean.\nFirst, let’s look at the landscape of how data engineering is described and develop\nsome terminology we can use throughout this book. Endless definitions of data\nengineering exist. In early 2022, a Google exact-match search for “what is data engi‐\nneering?” returns over 91,000 unique results. Before we give our definition, here are a\nfew examples of how some experts in the field define data engineering:\nData engineering is a set of operations aimed at creating interfaces and mechanisms\nfor the flow and access of information. It takes dedicated specialists—data engineers—\nto maintain data so that it remains available and usable by others. In short, data\nengineers set up and operate the organization’s data infrastructure, preparing it for\nfurther analysis by data analysts and scientists.\n3'

In [26]:
# faise database
from langchain_community.vectorstores import FAISS
embedding = OllamaEmbeddings(model="nomic-embed-text")
db1=FAISS.from_documents(documents[:100],embedding)

In [27]:
query="what is data engineering"
result=db1.similarity_search(query)
result[0].page_content

'rise of data science in the 2010s. For the purpose of this book, it’s critical to define\nwhat data engineering and data engineer mean.\nFirst, let’s look at the landscape of how data engineering is described and develop\nsome terminology we can use throughout this book. Endless definitions of data\nengineering exist. In early 2022, a Google exact-match search for “what is data engi‐\nneering?” returns over 91,000 unique results. Before we give our definition, here are a\nfew examples of how some experts in the field define data engineering:\nData engineering is a set of operations aimed at creating interfaces and mechanisms\nfor the flow and access of information. It takes dedicated specialists—data engineers—\nto maintain data so that it remains available and usable by others. In short, data\nengineers set up and operate the organization’s data infrastructure, preparing it for\nfurther analysis by data analysts and scientists.\n3'

In [33]:
from langchain_community.llms import Ollama
llm = Ollama(model='llama3')
llm

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10932\4265366075.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model='llama3')


Ollama(model='llama3')

In [37]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(""" 
answer the following question based only on the provided context.
                                        Think step by step before providing a detailed answer i will tip you $1000 if the user find the answer helful
                                        <context>
                                        {context}
                                        </context>
                                        Question:{input}""")
retriever=db.as_retriever

In [38]:
## Chain Introduction
## Create Stuff Document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
documents_chain=create_stuff_documents_chain(llm,prompt)